# CNA MAP
---

In [1]:
from IPython.core.magic import register_cell_magic
from IPython.display import Markdown
import datetime
from datetime import date
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
import json
import pandas as pd
import time

In [2]:


with open('CNAsList.json') as f:
    data = json.load(f)

CNA = pd.json_normalize(data)

geolocator = Nominatim(user_agent="CNALookup")
latitude = []
long = []
for i in CNA["country"]:
    if i != None:
        location = geolocator.geocode(i, timeout=None)
        if location!=None:
            latitude.append(location.latitude)#, location.longitude)
            long.append(location.longitude)
        else:
            latitude.append(float("Nan"))#, location.longitude)
            long.append(float("Nan"))
    else:
        latitude.append(float("Nan"))#, location.longitude)
        long.append(float("Nan"))

CNA["Latitude"] = latitude
CNA["Longitude"] = long

In [3]:
pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}[-a-zA-Z0-9()@:%_+.~#?&/=]*)' 
CNA['securityAdvisories.advisories'] = CNA['securityAdvisories.advisories'].astype(str)
CNA['URL'] = CNA['securityAdvisories.advisories'].str.extract(pattern, expand=False).str.strip()
CNA['URL'] = CNA['URL'].astype(str)


In [4]:
map = folium.Map(location=[10, 0], tiles='cartodb positron', zoom_start=2.25)

mcluster = MarkerCluster().add_to(map)

for index, row in CNA.iterrows():
    folium.Marker(location=[row['Latitude'],row['Longitude']], popup=("<b>"+row.organizationName+"</b>" +"\n"+"<br>"+row.URL),icon=folium.Icon(color='lightgray', icon='bug', prefix='fa')).add_to(mcluster)

map


This map is built from the data on [this page](https://www.cve.org/PartnerInformation/ListofPartners). More infomration about CNA's can be found [here](https://www.cve.org/ProgramOrganization/CNAs).  

In [5]:
Markdown(f"This report is updated automatically every day, last generated on: **{datetime.datetime.now()}**")

This report is updated automatically every day, last generated on: **2024-02-01 08:16:46.363690**